# Tavily Web Search
- LLM에게 웹검색을 쥐어주자

In [1]:
%pip install -q langchain-tavily

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from dotenv import load_dotenv
from pprint import pprint

load_dotenv()

True

In [3]:
from langchain_tavily import TavilySearch   

search_tool = TavilySearch(
    max_results=5,
    topic="general",
    search_depth="basic",
    # time_range="day",
    # include_domains=None,
    # exclude_domains=None
)

In [4]:
pprint(search_tool.invoke({'query': '강남 신세계에서 밥먹을만한 곳 알려줘'}))

{'answer': None,
 'follow_up_questions': None,
 'images': [],
 'query': '강남 신세계에서 밥먹을만한 곳 알려줘',
 'request_id': 'fa117fd8-7496-479a-872a-40a1433aa1cc',
 'response_time': 0.87,
 'results': [{'content': '강남신세계 혼밥맛집 (106곳) · 1. 소녀방앗간 서울고속터미널점 · 2. 타코벨 '
                         '서울고속버스터미널점 · 3. 소이연남 파미에스테이션점 · 4. 갓덴스시 센트럴시티점 · 5. '
                         '101번지',
              'raw_content': None,
              'score': 0.72231215,
              'title': "'강남신세계 혼밥' 맛집 빅데이터 추천순위 Top100 - 다이닝코드",
              'url': 'https://www.diningcode.com/list.dc?query=%EA%B0%95%EB%82%A8%EC%8B%A0%EC%84%B8%EA%B3%84%20%ED%98%BC%EB%B0%A5'},
             {'content': 'Missing: 밥 알려줘',
              'raw_content': None,
              'score': 0.71948266,
              'title': '강남 신세계 푸드코트 하우스오브 신세계,스위트파크(Eng CC)/ Seoul ...',
              'url': 'https://www.youtube.com/watch?v=wRvUnu8OwWI'},
             {'content': 'Missing: 밥 알려줘',
              'raw_content': None,
              'score': 0.694241,
  

In [5]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

llm = ChatOpenAI(model='gpt-4.1-nano')

prompt = PromptTemplate(
    input_variables=['query', 'search_results'],
    template="""넌 웹 검색결과를 요약해서 사용자의 질문에 맞게 좋은 답을 해주는 챗봇이야.

    질문: {query}
    검색결과: {search_results}
    질문에 맞는 답을 검색결과 바탕으로 간단하게 알려줘
    """
)

chain = (
    {
        'query': RunnablePassthrough(),  # 사용자 입력 그대로 이 자리에 들어옴
        'search_results': search_tool | RunnableLambda(
            lambda x: '\n'.join(
                [f'-{r['title']} ({r['content']})' for r in x['results']]
            )
        )
    }
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke('강남 신세계에서 점심 먹기 좋은곳')

'강남 신세계백화점 내에서 점심 식사를 찾으신다면, 하우스 오브 신세계의 타치바나 일본라멘이 추천됩니다. 평일 저녁 5시 30분 전에 가면 웨이팅이 적어 편리하며, 점심 시간에도 맛있는 일본 라멘을 즐기실 수 있습니다. 또한 신세계백화점의 푸드코트에서 다양한 맛집을 경험하실 수 있습니다.'

In [6]:
dumb_chain = prompt | llm | StrOutputParser()

dumb_chain.invoke({'query': '강남 신세계에서 점심먹기 좋은곳', 'search_results': ''})

'강남 신세계에서 점심을 먹기 좋은 곳으로는 다양한 레스토랑과 카페가 있습니다. 특히, 신세계백화점 내에는 일식, 양식, 한식 등 다양한 맛집이 있어 선택하기 좋아요. 참고로, 신세계 내에 위치한 한식당이나 뷔페도 인기입니다. 추가로 원하는 메뉴나 분위기를 알려주시면 더 구체적인 추천을 드릴 수 있어요.'

In [7]:
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from datetime import datetime

today = datetime.today().strftime('%D')

search_tool = TavilySearch(
    max_results=5,
    topic='general'
)

prompt = ChatPromptTemplate.from_messages([
    ('system', f'너는 훌륭한 어시스턴트야. 반드시 웹 검색도구를 사용해서 정보를 얻어. 최대한 답을 잘 해보자. 오늘은 {today}야.'),
    MessagesPlaceholder(variable_name='messages'),
    MessagesPlaceholder(variable_name='agent_scratchpad')  # 도구(검색) 호출때 필요함
])

agent = create_openai_tools_agent(
    llm=llm,
    tools=[search_tool],
    prompt=prompt
)

agent_executor = AgentExecutor(agent=agent, tools=[search_tool], verbose=True)

agent_executor.invoke({'messages': [{'role': 'human', 'content': '오늘 날씨에 맞는 음악 추천해줘'}]})



> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `{'query': '09/03/25 오늘 날씨'}`


{'query': '09/03/25 오늘 날씨', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://news.nate.com/view/20250903n17432', 'title': '오늘의 날씨 (25. 09. 03. 10시)', 'content': '낮 최고기온은 서울과 전주 33도, 대전과 대구는 32도까지 오르겠고요. 동풍이 부는 동해안 지역은 강릉 31도, 포항은 29도에 그치겠습니다. 남은 한 주간 평년기온을 웃', 'score': 0.90039873, 'raw_content': None}, {'url': 'http://www.signalnews.co.kr/1987013', 'title': "'오늘의 날씨' 25년 9월 3일", 'content': '남부지방을 중심으로 소나기가 내리는 곳이 있겠다. <기온> 오늘(3일) 낮 기온은 27~33℃로 어제(2일, 28.1~33.4℃)와 비슷하겠다.', 'score': 0.88799405, 'raw_content': None}, {'url': 'https://news.nate.com/view/20250903n01877', 'title': '오늘의 날씨 - 오전 (2025년 09월 03일)', 'content': '9월 3일 수요일, 아침 날씨입니다. 아침 최저기온은 서울 23.0도, 인천 22.0도, 수원 21.0도, 춘천 21.0도, 강릉 22.0도, 청주 22.0도, 대전 21.0도,', 'score': 0.8708723, 'raw_content': None}, {'url': 'https://www.chosun.com/national/transport-environment/2025/09/02/FZYFV7E3JBAYLG

{'messages': [{'role': 'human', 'content': '오늘 날씨에 맞는 음악 추천해줘'}],
 'output': '오늘(09/03/25)의 날씨는 대체로 맑거나 구름이 끼는 가운데, 낮 최고 기온이 서울과 전주에서는 33도, 대전과 대구는 32도까지 오르고, 강릉은 31도, 포항은 29도로 매우 더운 편입니다. 아침 기온은 서울 23도, 인천 22도, 춘천 21도 등으로 쾌적한 시작을 할 수 있겠습니다.\n\n이런 따뜻하고 화창한 날씨에는 상쾌하고 활기찬 음악을 추천드릴게요.\n\n### 오늘 날씨에 어울리는 음악 추천\n1. **여름 느낌의 팝곡**: 태양과 따뜻한 분위기를 떠오르게 하는 곡들  \n   - "Good as Hell" - Lizzo  \n   - "Uptown Funk" - Mark Ronson ft. Bruno Mars  \n\n2. **경쾌하고 활기찬 곡**: 산책이나 야외 활동에 적합한 곡들  \n   - "Walking on Sunshine" - Katrina & The Waves  \n   - "Happy" - Pharrell Williams  \n\n3. **여름과 자연을 연상시키는 음악**: 자연의 소리나 편안한 멜로디  \n   - "Island in the Sun" - Weezer  \n   - "Send Me On My Way" - Rusted Root  \n\n이 음악들로 오늘의 활기찬 하루를 더욱 즐겁게 보내보세요!'}